In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yaml,f90nml,sys,os,git
from pathlib import Path
from yaml.representer import SafeRepresenter

dir_ensemble = '/g/data/tm70/ml0072/COMMON/ensemble'
sys.path.append(dir_ensemble)
os.chdir(dir_ensemble)
os.getcwd()
print(sys.version)

3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]


In [3]:
sys.path.append('/g/data/tm70/ml0072/COMMON/om3-utils/om3utils')

In [13]:
mom6_folder = '/g/data/tm70/ml0072/MOM6-CICE6/PROFILING/3_2using_scripts/clx-run-o1344i96c96d48-fcpl-s-fm-F-fr-c_no_diags'
mom6_input = 'MOM_input'
input_path = os.path.join(mom6_folder,mom6_input)
from mom6_input import read_mom6_input,write_mom6_input,_patch_mom6_input_str
# read MOM6 input 
mom6_input = read_mom6_input(input_path)
print(type(mom6_input))
mom6_output = write_mom6_input(mom6_input,Path(mom6_folder+"/MOM_input_test5"))
mom6_output

# with open(input_path,'r') as f:
#     str_tmp1 = f.read()
# output,patch = _patch_mom6_input_str(str_tmp1)

<class 'mom6_input.Mom6Input'>


In [47]:
patch

{153: ('zstar',
  '                                !  ZSTAR, Z* - stretched geopotential z*\n'),
 467: ('block', 'KPP%\n'),
 481: ('block', '%KPP\n'),
 488: ('block',
  '                                ! parameters are contained in the CVMix_CONVECTION% parameter block.\n'),
 489: ('block', 'CVMix_CONVECTION%\n'),
 490: ('block', '%CVMix_CONVECTION\n'),
 533: ('block', 'CVMIX_DDIFF%\n'),
 534: ('block', '%CVMIX_DDIFF\n')}

In [80]:
from pprint import pprint
#pprint(output

nml_str = "&mom6\n" + output + "\n/"
#pprint(nml_str)
parser = f90nml.Parser()
nml = parser.reads(nml_str)
#nml
tmp_dict = dict(nml.todict()['mom6'])
for key in list(tmp_dict.keys()):
    if not key.isupper():
        tmp_dict[key.upper()] = tmp_dict.pop(key)

tmp_dict

{'DIABATIC_FIRST': True,
 'USE_REGRIDDING': True,
 'THICKNESSDIFFUSE': True,
 'THICKNESSDIFFUSE_FIRST': True,
 'DT': 1350.0,
 'DT_THERM': 1350.0,
 'HFREEZE': 10.0,
 'DTBT_RESET_PERIOD': 0.0,
 'FRAZIL': True,
 'BOUND_SALINITY': True,
 'C_P': 3992.0,
 'CHECK_BAD_SURFACE_VALS': True,
 'BAD_VAL_SSH_MAX': 50.0,
 'BAD_VAL_SSS_MAX': 75.0,
 'BAD_VAL_SST_MAX': 55.0,
 'BAD_VAL_SST_MIN': -3.0,
 'SAVE_INITIAL_CONDS': True,
 'TRIPOLAR_N': True,
 'NIGLOBAL': 1440,
 'NJGLOBAL': 1080,
 'INPUTDIR': './input/',
 'GRID_CONFIG': 'mosaic',
 'GRID_FILE': 'ocean_hgrid.nc',
 'TOPO_CONFIG': 'file',
 'MINIMUM_DEPTH': 0.5,
 'MAXIMUM_DEPTH': 6000.0,
 'CHANNEL_CONFIG': 'none',
 'NK': 50,
 'DTFREEZE_DP': -7.75e-08,
 'USE_IDEAL_AGE_TRACER': True,
 'REGRIDDING_COORDINATE_MODE': 'ZSTAR',
 'ALE_COORDINATE_CONFIG': 'FILE:ocean_vgrid.nc,interfaces=zeta',
 'REMAPPING_SCHEME': 'PPM_H4',
 'INIT_LAYERS_FROM_Z_FILE': True,
 'TEMP_SALT_Z_INIT_FILE': 'ocean_temp_salt.res.nc',
 'Z_INIT_FILE_PTEMP_VAR': 'temp',
 'Z_INIT_ALE_REMAP

In [19]:
import re

mom6_input = 'MOM_input'
input_path = os.path.join(mom6_folder,mom6_input)
with open(input_path,'r') as f:
    str_tmp1 = f.read()


# Define several patterns that need to be matched
comment_pattern = re.compile(r"/\*.*?\*/", flags=re.DOTALL)
zstar_pattern = re.compile(r"Z\*")
block_pattern = re.compile(r"KPP%|%KPP|CVMix_CONVECTION%|%CVMix_CONVECTION|CVMIX_DDIFF%|%CVMIX_DDIFF")
override_directive_pattern = re.compile(r"^(#override\s*?)")
incorrect_directive_pattern = re.compile(r"^(#\s+)")
comment_directive_pattern = re.compile(r"^#((?!override)\w+\b\s*=\s*\w+$)")

# Modify the input while recording the changes
patch = {}
output = ""
lines = str_tmp1.split("\n")
for i in range(len(lines)):
    line = lines[i] + "\n"
    if zstar_pattern.search(line):
        patch[i] = ("zstar", line)
        output += zstar_pattern.sub("ZSTAR", line)
    elif block_pattern.search(line):
        patch[i] = ("block", line)
        output += block_pattern.sub("", line)
        
    elif override_directive_pattern.search(line):
        patch[i] = ("override", override_directive_pattern.match(line).group(0))
        output += override_directive_pattern.sub("", line)
    elif incorrect_directive_pattern.search(line):
        patch[i] = (
            "incorrect directive",
            incorrect_directive_pattern.match(line).group(0),
        )
        output += incorrect_directive_pattern.sub("", line)
    elif comment_directive_pattern.search(line):
        patch[i] = ("comment_directive", line)
        output += "\n"
    else:
        output += line

    #print(output)


KPP%

%KPP

                                ! parameters are contained in the CVMix_CONVECTION% parameter block.

CVMix_CONVECTION%

%CVMix_CONVECTION

CVMIX_DDIFF%

%CVMIX_DDIFF



In [22]:
patch

test = 'CVMix_CONVECTION%'
test = '%CVMix_CONVECTION'
test.isupper()

False

In [35]:
mom6_input = 'MOM_input_test'
input_path = os.path.join(mom6_folder,mom6_input)
with open(input_path,'r') as f:
    str_tmp1 = f.read()

str_tmp1
lines = str_tmp1.split("\n")
lines
# for i in range(len(lines)):
#     line = lines[i] +'\n' d
    
# comment_pattern = re.compile(r"/\*.*?\*/", flags=re.DOTALL)
# comment_pattern

['! This file was written by the model and records the non-default parameters used at run-time.',
 '',
 '! === module MOM ===',
 'DIABATIC_FIRST = True           !   [Boolean] default = False',
 '! If true, apply diabatic and thermodynamic processes, including buoyancy',
 '! forcing and mass gain or loss, before stepping the dynamics forward.',
 'USE_REGRIDDING = True           !   [Boolean] default = False',
 '! If True, use the ALE algorithm (regridding/remapping). If False, use the',
 '! layered isopycnal algorithm.',
 'THICKNESSDIFFUSE = True         !   [Boolean] default = False',
 '! If true, isopycnal surfaces are diffused with a Laplacian coefficient of KHTH.',
 'THICKNESSDIFFUSE_FIRST = True   !   [Boolean] default = False',
 '! If true, do thickness diffusion or interface height smoothing before dynamics.',
 '! This is only used if THICKNESSDIFFUSE or APPLY_INTERFACE_FILTER is true.',
 'DT = 1350.0                     !   [s]',
 '! The (baroclinic) dynamics time step.  The ti

In [22]:
yamlfile = 'ensemble.yaml'
from ensemble import ensemble

NameError: name 'yamlfile' is not defined